In [1]:
!pip install qdrant-client groq sentence-transformers dspy-ai fastembed gradio langchain langchain-google-genai PyPDF2 --upgrade --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.0/55.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.0/990.0 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [2]:
from qdrant_client import QdrantClient
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import dotenv

In [5]:
from PyPDF2 import PdfReader
def get_pdf_text(pdf_path):
    text = ""
    pdf_reader = PdfReader(pdf_path)
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text

# Verify and correct the file path
pdf_path = "/content/data/sodapdf-converted.pdf"

# Check if the file exists
text = get_pdf_text(pdf_path)
print("done")

done


In [6]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=102)
    chunks = text_splitter.split_text(text)
    return chunks

chunks = get_text_chunks(text)
print("done")

done


In [7]:
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("BAAI/bge-large-en-v1.5", device='cpu')
vectors = []
for chunk in tqdm(chunks, desc="Encoding chunks"):
    vector = model.encode(chunk)
    vectors.append(vector)

print("done")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Encoding chunks: 100%|██████████| 588/588 [14:06<00:00,  1.44s/it]

done


In [8]:
vectors[0].shape

(1024,)

In [16]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
from google.colab import userdata
qdrant_api_key =userdata.get('qdrant_api_key')

qdrant_client = QdrantClient(
    url="https://1947bd36-6e0c-4cdf-bef2-4d589fdff1ef.us-east4-0.gcp.cloud.qdrant.io:6333",
    api_key=qdrant_api_key,
)

qdrant_client.recreate_collection(
    collection_name="dspy-application",
    vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
)

<ipython-input-16-7d75bd3dee01>:11: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


True

In [17]:
qdrant_client.upload_collection(
    collection_name="dspy-application",
    ids=[i for i in range(len(chunks))],
    vectors=vectors,
    parallel=4,
    max_retries=3,
)

In [18]:
print(qdrant_client.get_collections())

collections=[CollectionDescription(name='dspy-application')]


In [19]:
from dspy.retrieve.qdrant_rm import QdrantRM
qdrant_retriever_model = QdrantRM("dspy-application", qdrant_client, k=3)

100%|██████████| 77.7M/77.7M [00:06<00:00, 12.5MiB/s]


In [20]:
import dspy
groq_api_key =userdata.get('groq_api_key')
llama3 = dspy.GROQ(model='llama3-8b-8192', api_key = groq_api_key )

In [26]:
dspy.settings.configure(rm=qdrant_retriever_model, lm=llama3)
class GenerateAnswer(dspy.Signature):
    """Answer questions with logical factoid answers."""

    context = dspy.InputField(desc="will contain a princess and wizard related story")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="a detailed answer")

In [27]:
def get_context(text):
    query_vector = model.encode(text)


    hits = qdrant_client.search(
        collection_name="dspy-application",
        query_vector=query_vector,
        limit=3  # Return 5 closest points
    )
    s=''
    for x in [chunks[i.id] for i in hits]:
        s = s + x
    return s

In [28]:
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()


        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)


    def forward(self, question):
        context = get_context(question)
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

In [31]:
rag = RAG()
def respond(query, chat_history=None): # Add chat_history with a default value
    response = rag(query)
    return response.answer

answer  = respond("explain me what happens in the chapter 2") # Call without chat_history, using the default
print(answer)

In Chapter 2, the enemy is hesitating to renew the attack, and the Wizard's revolver is making them retreat. Zeb is confident that they have the enemy on the run, but the Wizard is gloomy because they only have six shots left before they become helpless. Meanwhile, a kitten is accused of stealing a piglet and hiding it, and the Princess Oz


In [34]:
import gradio as gr

# Define the respond function
def respond(query, chat_history=None):
    response = rag(query)
    if chat_history is None:
        chat_history = []
    chat_history.append((query, response.answer))
    return "", chat_history

# Create the Gradio interface
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

# Launch the Gradio demo
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://928c329b8b81791dae.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
